In [1]:
import os
from typing import List
import json
import argparse
import logging
import pandas as pd
from tqdm import tqdm
import evaluate
# Load the BLEU and ROUGE metrics
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load('meteor')

/Users/taindp/miniconda3/envs/hipporag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package wordnet to /Users/taindp/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/taindp/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/taindp/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
from rank_bm25 import BM25Okapi

In [3]:
def compute_recall_at_k(groundtruths, prediction_ids, k=5):
    """
    groundtruths: list of ground truth labels (length N)
    prediction_ids: list of lists, each inner list contains top-k predictions for a sample (length N x k)
    k: number of top predictions to consider (default 5)
    """
    correct = 0
    total = len(groundtruths)
    for gt, preds in zip(groundtruths, prediction_ids):
        if gt in preds[:k]:
            correct += 1
    recall_at_k = correct / total if total > 0 else 0
    return recall_at_k

In [6]:
major = "MCS"
topk = 5
corpus_path = f"../data/courses_{major}.json"
with open(corpus_path, "r") as f:
    corpus = json.load(f)

docs = [f"{doc['title']}\n{doc['text']}" for doc in corpus]

tokenized_corpus = [doc.split(" ") for doc in docs]
bm25 = BM25Okapi(tokenized_corpus)
open_end_qa_ds = pd.DataFrame(json.load(open(f"../data/{major}_opened_end.json", "r")))
queries = open_end_qa_ds["question"].tolist()
references = open_end_qa_ds["answer"].tolist()
groundtruths = [item[0]["title"] for item in open_end_qa_ds["paragraphs"].tolist()]
predictions = [bm25.get_top_n(query.split(" "), docs, n=topk) for query in tqdm(queries)]
prediction_ids = []
for sublist in predictions:
    tmp = []
    for item in sublist:
        tmp.append(item.split("\n")[0])
    prediction_ids.append(tmp)
recall = compute_recall_at_k(groundtruths=groundtruths, prediction_ids=prediction_ids, k=topk)
print(f"Recall@5: {recall:.4f}")

100%|██████████| 25/25 [00:00<00:00, 1831.09it/s]

Recall@5: 0.7600
